In [15]:
import json
from sklearn.metrics import cohen_kappa_score
import numpy as np

In [16]:
# Load the annotations from JSON file
def load_annotations(file_path):
    with open(file_path, 'r') as jfile:
        jdata = json.load(jfile)
    return jdata

In [17]:
# Extract annotations from JSON file
def extract_annotations(annotations):
    annot_dict = {}
    for task in annotations:
        full_text = task['data']['text']
        annot_dict[task['id']] = {'text': full_text, 'overall_label': None, 'span_labels': {}}
        # Extarct annot info
        for annot in task['annotations'][0]['result']:
            if annot['from_name'] == 'label':
                start = annot['value']['start']
                end = annot['value']['end']
                text = annot['value']['text'].strip()
                label = annot['value']['labels'][0]
                annot_dict[task['id']]['span_labels'][(start, end)] = (text, label)
            else:
                annot_dict[task['id']]['overall_label'] = annot['value']['choices'][0]
    return annot_dict

In [18]:
def extract_token_labels(annotations, text_length):
    """Extract token labels for the full text with 'O' for non-annotated spans."""
    token_labels = ['O'] * text_length  # Initialize all tokens as unlabeled ('O')
    for (start, end), (entity_text, label) in annotations['span_labels'].items():
        for i in range(start, end):  # Assign label to each character in the span
            token_labels[i] = label
    return token_labels

def calculate_kappa(annotations_1, annotations_2):
    """Calculate Cohen's Kappa score using the full text provided in annotations."""
    kappa_scores = {}
    
    for task_id in annotations_1.keys():
        if task_id in annotations_2:
            # Get text length from full text
            text_length = len(annotations_1[task_id]['text'])

            # Extract token labels for both annotators
            labels_1 = extract_token_labels(annotations_1[task_id], text_length)
            labels_2 = extract_token_labels(annotations_2[task_id], text_length)

            # Calculate Kappa for the current task
            kappa_scores[task_id] = cohen_kappa_score(labels_1, labels_2)

    mean_score = np.mean(list(kappa_scores.values()))
    return mean_score

In [19]:
def calculate_psa(annotations_1, annotations_2):
    """Calculate PSA using the full text provided in annotations."""
    psa_scores = {}

    for task_id in annotations_1.keys():
        if task_id in annotations_2:
            # Get text length from full text
            text_length = len(annotations_1[task_id]['text'])

            # Extract token labels for both annotators
            labels_1 = extract_token_labels(annotations_1[task_id], text_length)
            labels_2 = extract_token_labels(annotations_2[task_id], text_length)

            # Identify positively labeled tokens (ignoring 'O')
            labeled_1 = {i for i, label in enumerate(labels_1) if label != 'O'}
            labeled_2 = {i for i, label in enumerate(labels_2) if label != 'O'}

            # Calculate intersection and total positives
            intersection = len(labeled_1 & labeled_2)
            total_positive = len(labeled_1) + len(labeled_2)

            if total_positive == 0:
                psa_scores[task_id] = 1.0  # If no positive labels, PSA is 1
            else:
                psa_scores[task_id] = (2 * intersection) / total_positive

    mean_score = np.mean(list(psa_scores.values()))
    return mean_score

Labeling 1-100

In [20]:
iza_1_100 = load_annotations('1-100/iza_1_100.json')
filip_1_100 = load_annotations('1-100/filip_1_100.json')

In [21]:
iza_annots = extract_annotations(iza_1_100)
filip_annots = extract_annotations(filip_1_100)

In [22]:
kappa = calculate_kappa(iza_annots, filip_annots)
print(f"Mean Cohen's Kappa: {kappa}")

# Calculate Positive Specific Agreement (PSA)
psa = calculate_psa(iza_annots, filip_annots)
print(f"Mean Positive Specific Agreement: {psa}")

Mean Cohen's Kappa: 0.7676451708357998
Mean Positive Specific Agreement: 0.8136569537176238
